# Sentiment Graph by Region

Author: @SirenaYu

This notebook is divided into 2 sections:
* Section 1 is data exploration, in which we identify the top 10 cities for each country covered by Twitter geography data.
* Section 2 is graph generation, in which time series graphs from 2015 to 2021 is generated for each region.

This is a little messy, to be organized.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import gzip
from script import days_in_month
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from datetime import date as dt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Data Exploration

## 2015/1/10, 0AM Data 

In [ ]:
geo_path = "/srv/data/twitter_geography/2015/"
sent_path = "/srv/data/twitter_sentiment/2015/"

with gzip.open(''.join([geo_path, "geography_2015_1_10_00.csv.gz"])) as f:
    geo_posts = pd.read_csv(f, sep="\t")

with gzip.open(''.join([sent_path, "bert_sentiment_2015_1_10_00.csv.gz"])) as f:
    sent_posts = pd.read_csv(f, sep="\t")
    
common_posts = pd.merge(geo_posts, sent_posts, on="message_id", how="inner")

common_posts.head()

In [ ]:
len(common_posts)

In [ ]:
common_posts["NAME_0"].value_counts().nlargest(10)

In [ ]:
top_5_us_cities = common_posts[common_posts["NAME_0"] == "United States"]["NAME_2"].value_counts().nlargest(5).index.tolist()
top_5_us_cities

In [ ]:
city_to_sent_score = dict()

for city in top_5_us_cities:
    avg_sent_score = common_posts[common_posts["NAME_2"] == city]["score"].mean()
    city_to_sent_score[city] = avg_sent_score
        
city_to_sent_score

## 2015/1/10, All Day Data

To calculate the daily average of top 5 US cities, we do the following:

* For each hour of the day, keep track of the top 10 US cities and their average sentiment scores / number of posts.
* Sum up number of posts and calculate top 5 US cities of that day.
* Using average sentiment scores / number of posts from each hour, calculate a weighted average, resulting in an accurate daily average.

In [ ]:
geo_path = "/srv/data/twitter_geography/2015/"
sent_path = "/srv/data/twitter_sentiment/2015/"

In [ ]:
for hour in range(0, 1):
    with gzip.open(''.join([geo_path, "geography_2015_1_10_", str(hour).zfill(2), ".csv.gz"])) as f:
        geo_posts = pd.read_csv(f, sep="\t")
    with gzip.open(''.join([sent_path, "bert_sentiment_2015_1_10_", str(hour).zfill(2), ".csv.gz"])) as f:
        sent_posts = pd.read_csv(f, sep="\t")
    common_posts = pd.merge(geo_posts, sent_posts, on="message_id", how="inner")
    

In [ ]:
common_posts.head()

In [ ]:
common_posts["STATE_CITY"] = common_posts['NAME_1'].astype(str) + '_' + common_posts['NAME_2']
common_posts.head()

In [ ]:
common_posts[common_posts["STATE_CITY"]=="Arizona_Maricopa"]

In [ ]:
geo_path = "/srv/data/twitter_geography/2015/"
sent_path = "/srv/data/twitter_sentiment/2015/"

hour_2_city_2_avg_n_numpost = dict()
city_2_numpost = dict()
city_avg_score_numpost_df = None

for hour in range(0, 24):
    with gzip.open(''.join([geo_path, "geography_2015_1_10_", str(hour).zfill(2), ".csv.gz"])) as f:
        geo_posts = pd.read_csv(f, sep="\t")
    with gzip.open(''.join([sent_path, "bert_sentiment_2015_1_10_", str(hour).zfill(2), ".csv.gz"])) as f:
        sent_posts = pd.read_csv(f, sep="\t")
    common_posts = pd.merge(geo_posts, sent_posts, on="message_id", how="inner")
    top_10_us_cities = common_posts[common_posts["NAME_0"] == "United States"]["NAME_2"].value_counts().nlargest(10).index.tolist()
    city_2_avg_n_numpost = dict()
    post_in_cities = common_posts[common_posts["NAME_2"].isin(top_10_us_cities)]
    city_result = post_in_cities.groupby(["NAME_2"]).agg({"score": np.sum, "message_id": len}).reset_index()
    city_result.rename(columns={"NAME_2": "city", "score": "total_score", "message_id": "num_posts"}, inplace=True)
    if city_avg_score_numpost_df is None:
        city_avg_score_numpost_df = city_result
    else:
        city_avg_score_numpost_df = city_avg_score_numpost_df.merge(city_result, on="city", how="outer", suffixes=('_x', '_y'))
        city_avg_score_numpost_df["total_score"] = city_avg_score_numpost_df["total_score_x"].fillna(0) + city_avg_score_numpost_df["total_score_y"].fillna(0)
        city_avg_score_numpost_df["num_posts"] = city_avg_score_numpost_df["num_posts_x"].fillna(0) + city_avg_score_numpost_df["num_posts_y"].fillna(0)
        city_avg_score_numpost_df.drop(columns=["total_score_x", "total_score_y", "num_posts_x", "num_posts_y"], inplace=True)
    hour_2_city_2_avg_n_numpost[hour] = city_result
    

In [ ]:
city_avg_score_numpost_df = city_avg_score_numpost_df.sort_values(by=['num_posts'], ascending=False).reset_index(drop=True)[:5]
city_avg_score_numpost_df

In [ ]:
city_avg_score_numpost_df["daily_avg_score"] = city_avg_score_numpost_df["total_score"]/city_avg_score_numpost_df["num_posts"]

In [ ]:
city_avg_score_numpost_df

In [ ]:
top_5_us_cities = [k for k, v in sorted(city_2_numpost.items(), key=lambda item: item[1], reverse=True)][:5]
top_5_us_cities

In [ ]:
city_2_avg = dict()

for city in top_5_us_cities:
    for hour in range(0, 24):
        try: 
            hour_avg, hour_numpost = hour_2_city_2_avg_n_numpost[hour][city]
            weight_avg = hour_avg * hour_numpost / city_2_numpost[city]
            if city not in city_2_avg.keys():
                city_2_avg[city] = weight_avg
            else:
                city_2_avg[city] += weight_avg
        except KeyError:
            continue

city_2_avg

In [ ]:
def get_daily_sent_avg_n_numpost_top_m_cities(country, m, year, month, day):
    """
    @param country: str
    @param m: int, top m cities that we want to look at
    @param year: int
    @param month: int
    @param day: int
    
    return: dict, maps city to (daily_sent_avg, daily_numpost)
    """
    start_time = datetime.now()
    geo_path = "".join(["/srv/data/twitter_geography/", str(year), "/"])
    sent_path = "".join(["/srv/data/twitter_sentiment/", str(year), "/"])
    date = "".join([str(year), "_", str(month), "_", str(day).zfill(2)])

    
    result_df = None

    for hour in range(0, 24):
        pre_open_time = datetime.now()
        try:
            with gzip.open(''.join([geo_path, "geography_", date, "_", str(hour).zfill(2), ".csv.gz"])) as f:
                geo_posts = pd.read_csv(f, sep="\t")
        except FileNotFoundError:
            print(''.join([geo_path, "geography_", date, "_", str(hour).zfill(2), ".csv.gz"]), "does not exist.")
            continue
        except pd.errors.EmptyDataError:
            print(''.join([geo_path, "geography_", date, "_", str(hour).zfill(2), ".csv.gz"]), "is empty.")
            continue
        try:
            with gzip.open(''.join([sent_path, "bert_sentiment_", date, "_", str(hour).zfill(2), ".csv.gz"])) as f:
                sent_posts = pd.read_csv(f, sep="\t")
        except FileNotFoundError:
            print(''.join([sent_path, "bert_sentiment_", date, "_", str(hour).zfill(2), ".csv.gz"]), "does not exist.")
            continue
        except pd.errors.EmptyDataError:
            print(''.join([sent_path, "bert_sentiment_", date, "_", str(hour).zfill(2), ".csv.gz"]), "is empty.")
            continue
        
        common_posts = pd.merge(geo_posts, sent_posts, on="message_id", how="inner")
        
        top_2m_cities = common_posts[common_posts["NAME_0"] == country]["NAME_2"].value_counts().nlargest(2*m).index.tolist()
        post_in_cities = common_posts[common_posts["NAME_2"].isin(top_2m_cities)]
        city_result = post_in_cities.groupby(["NAME_2"]).agg({"score": np.sum, "message_id": len}).reset_index()
        city_result.rename(columns={"NAME_2": "city", "score": "total_score", "message_id": "num_posts"}, inplace=True)
        if result_df is None:
            result_df = city_result
        else:
            result_df = result_df.merge(city_result, on="city", how="outer", suffixes=('_x', '_y'))
            result_df["total_score"] = result_df["total_score_x"].fillna(0) + result_df["total_score_y"].fillna(0)
            result_df["num_posts"] = result_df["num_posts_x"].fillna(0) + result_df["num_posts_y"].fillna(0)
            result_df.drop(columns=["total_score_x", "total_score_y", "num_posts_x", "num_posts_y"], inplace=True)
        
    result_df = result_df.sort_values(by=['num_posts'], ascending=False).reset_index(drop=True)[:m]
    result_df["daily_avg_score"] = result_df["total_score"]/result_df["num_posts"]
    result_df.drop(columns=["total_score"], inplace=True)
    end_time = datetime.now()
    print("get_daily_sent_avg_n_numpost_top_m_cities took", end_time-start_time, "seconds.")
    return result_df


In [ ]:
get_daily_sent_avg_n_numpost_top_m_cities("United States", 5, 2015, 1, 10)

passed test case 1!

In [ ]:
get_daily_sent_avg_n_numpost_top_m_cities("United States", 15, 2016, 1, 10)

In [ ]:
get_daily_sent_avg_n_numpost_top_m_cities("United Kingdom", 15, 2016, 1, 10)

In [ ]:
get_daily_sent_avg_n_numpost_top_m_cities("United States", 15, 2019, 1, 10)

# Graph Generation

## 2019/1, Daily Average, United States

In [ ]:
result_df = None

for year in range(2019, 2020):
    for month in range(1, 2):
        for day in range(1, days_in_month(month, year)+1):
            date = "".join([str(year), "_", str(month), "_", str(day).zfill(2)])
            daily_df = get_daily_sent_avg_n_numpost_top_m_cities("United States", 15, year, month, day)
            print(date)
            daily_df = daily_df.rename(columns={"daily_avg_score": date})
            display(daily_df)
            if result_df is None:
                result_df = daily_df
            else:
                result_df = result_df.merge(daily_df, on="city", how="outer", suffixes=('_x', '_y'))
                result_df["num_posts"] = result_df["num_posts_x"].fillna(0) + result_df["num_posts_y"].fillna(0)
                result_df.drop(columns=["num_posts_x", "num_posts_y"], inplace=True)
            display(result_df)

In [ ]:
result_df = result_df.sort_values(by=['num_posts'], ascending=False).reset_index(drop=True)[:5]
result_df

In [ ]:
result_df.drop(columns=["num_posts"], inplace=True)

In [ ]:
graph_df = result_df.T
graph_df.rename(columns=graph_df.iloc[0], inplace=True)
graph_df.drop(graph_df.index[0], inplace=True)

In [ ]:
graph_df

In [ ]:
graph_df.columns.to_list()

In [ ]:
for city in graph_df.columns.to_list():
    # df = pd.read_csv("".join(["../output/sentiment_graph_by_region/", city, "_2019_1.csv"]))
    base = dt(2019, 1, 1)
    numdays = 31
    x = [base + timedelta(days=x) for x in range(numdays)]
    y = graph_df[city]

    plt.plot(x, y)
    
    plt.title("".join(["Daily Average Sentiment of Posts in ", city, " County, 2019-1"]))
    plt.xticks(rotation = 45)
    plt.xlabel("Dates")
    plt.ylabel("Daily Average Sentiment")
    # plt.legend(bbox_to_anchor=(1.6, 1.0), loc='upper right')

    plt.show()
    plt.savefig("".join(["../output/sentiment_graph_by_region/", city, "_2019_1_graph.png"]))

## 2019, Daily Average, United States 

In [ ]:
def get_daily_sent_avg_n_numpost(country, cities, year, month, day):
    """
    @param county: str, desire country
    @param cities: list of str, cities that we want to look at
    @param year: int
    @param month: int
    @param day: int
    
    return: dict, maps city to (daily_sent_avg, daily_numpost)
    """
    start_time = datetime.now()
    geo_path = "".join(["/srv/data/twitter_geography/", str(year), "/"])
    sent_path = "".join(["/srv/data/twitter_sentiment/", str(year), "/"])
    date = "".join([str(year), "_", str(month), "_", str(day).zfill(2)])
    
    result_df = None

    for hour in range(0, 24):
        pre_open_time = datetime.now()
        try:
            with gzip.open(''.join([geo_path, "geography_", date, "_", str(hour).zfill(2), ".csv.gz"])) as f:
                geo_posts = pd.read_csv(f, sep="\t")
        except FileNotFoundError:
            print(''.join([geo_path, "geography_", date, "_", str(hour).zfill(2), ".csv.gz"]), "does not exist.")
            continue
        except pd.errors.EmptyDataError:
            print(''.join([geo_path, "geography_", date, "_", str(hour).zfill(2), ".csv.gz"]), "is empty.")
            continue
        try:
            with gzip.open(''.join([sent_path, "bert_sentiment_", date, "_", str(hour).zfill(2), ".csv.gz"])) as f:
                sent_posts = pd.read_csv(f, sep="\t")
        except FileNotFoundError:
            print(''.join([sent_path, "bert_sentiment_", date, "_", str(hour).zfill(2), ".csv.gz"]), "does not exist.")
            continue
        except pd.errors.EmptyDataError:
            print(''.join([sent_path, "bert_sentiment_", date, "_", str(hour).zfill(2), ".csv.gz"]), "is empty.")
            continue
        
        common_posts = pd.merge(geo_posts, sent_posts, on="message_id", how="inner")
        
        post_in_cities = common_posts[common_posts["NAME_2"].isin(cities)]
        city_result = post_in_cities.groupby(["NAME_2"]).agg({"score": np.sum, "message_id": len}).reset_index()
        city_result.rename(columns={"NAME_2": "city", "score": "total_score", "message_id": "num_posts"}, inplace=True)
        if result_df is None:
            result_df = city_result
        else:
            result_df = result_df.merge(city_result, on="city", how="outer", suffixes=('_x', '_y'))
            result_df["total_score"] = result_df["total_score_x"].fillna(0) + result_df["total_score_y"].fillna(0)
            result_df["num_posts"] = result_df["num_posts_x"].fillna(0) + result_df["num_posts_y"].fillna(0)
            result_df.drop(columns=["total_score_x", "total_score_y", "num_posts_x", "num_posts_y"], inplace=True)
    
    if result_df is None:
        result_data = np.array([
            cities,
            [0] * len(cities),
            [0] * len(cities)
        ])
        result_df = pd.DataFrame(data=result_data.T,
                                columns=["city", "num_posts", "daily_avg_score"])
        result_df = result_df.astype({'num_posts': 'int64', 'daily_avg_score': 'int64'})
    else:
        result_df["daily_avg_score"] = result_df["total_score"]/result_df["num_posts"]
        result_df.drop(columns=["total_score"], inplace=True)
    
    end_time = datetime.now()
    print("get_daily_sent_avg_n_numpost took", end_time-start_time, "seconds.")
    return result_df


In [ ]:
for year in range(2019, 2020):
    for month in range(1, 13):
        result_df = None
        for day in range(1, days_in_month(month, year)+1):
            date = "".join([str(year), "_", str(month), "_", str(day).zfill(2)])
            daily_df = get_daily_sent_avg_n_numpost("United States", cities, year, month, day)
            daily_df = daily_df.rename(columns={"daily_avg_score": date})
            if result_df is None:
                result_df = daily_df
            else:
                result_df = result_df.merge(daily_df, on="city", how="outer", suffixes=('_x', '_y'))
                # print(day, result_df.dtypes)
                result_df["num_posts"] = result_df["num_posts_x"].fillna(0) + result_df["num_posts_y"].fillna(0)
                result_df.drop(columns=["num_posts_x", "num_posts_y"], inplace=True)
        result_df.drop(columns=["num_posts"], inplace=True)
        graph_df = result_df.T
        graph_df.rename(columns=graph_df.iloc[0], inplace=True)
        graph_df.drop(graph_df.index[0], inplace=True)
        graph_df.to_csv("".join(["../output/sentiment_graph_by_region/us_top5_cities_2019_", str(month), ".csv"]))

In [ ]:
year_df = None

for month in range(1, 13):
    month_df = pd.read_csv("".join(["../output/sentiment_graph_by_region/us_top5_cities_2020_", str(month), ".csv"]))
    month_df = month_df.rename(columns={"Unnamed: 0":"date"})
    if year_df is None:
        year_df = month_df
    else:
        year_df = pd.concat([year_df, month_df], ignore_index=True)

year_df

In [ ]:
for year in range(2020, 2021):
    for month in range(1, 13):
        result_df = None
        for day in range(1, days_in_month(month, year)+1):
            date = "".join([str(year), "_", str(month), "_", str(day).zfill(2)])
            daily_df = get_daily_sent_avg_n_numpost("United States", cities, year, month, day)
            daily_df = daily_df.rename(columns={"daily_avg_score": date})
            if result_df is None:
                result_df = daily_df
            else:
                result_df = result_df.merge(daily_df, on="city", how="outer", suffixes=('_x', '_y'))
                # print(day, result_df.dtypes)
                result_df["num_posts"] = result_df["num_posts_x"].fillna(0) + result_df["num_posts_y"].fillna(0)
                result_df.drop(columns=["num_posts_x", "num_posts_y"], inplace=True)
        result_df.drop(columns=["num_posts"], inplace=True)
        graph_df = result_df.T
        graph_df.rename(columns=graph_df.iloc[0], inplace=True)
        graph_df.drop(graph_df.index[0], inplace=True)
        graph_df.to_csv("".join(["../output/sentiment_graph_by_region/us_top5_cities_2020_", str(month), ".csv"]))

In [ ]:
top_5_us_cities = ['Los Angeles', 'Harris', 'Cook', 'New York', 'Orange']

for city in top_5_us_cities:
    # df = pd.read_csv("".join(["../output/sentiment_graph_by_region/", city, "_2019_1.csv"]))
    base = dt(2020, 1, 1)
    numdays = 366
    x = [base + timedelta(days=x) for x in range(numdays)]
    y = year_df[city]

    plt.plot(x, y)
    
    plt.title("".join(["Daily Average Sentiment of Posts in ", city, " County, 2020"]))
    plt.xticks(rotation = 45)
    plt.xlabel("Dates")
    plt.ylabel("Daily Average Sentiment")
    # plt.legend(bbox_to_anchor=(1.6, 1.0), loc='upper right')

    plt.show()
    plt.savefig("".join(["../output/sentiment_graph_by_region/", city, "_2020_graph.png"]))

In [ ]:
def generate_daily_avg_df_by_year(country, cities, year):
    """
    @param cities: list of str, list of cities to generate 
    """
    result_df = None
    for month in range(1, 13):
        for day in range(1, days_in_month(month, year)+1):
            date = "".join([str(year), "_", str(month), "_", str(day).zfill(2)])
            daily_df = get_daily_sent_avg_n_numpost("United States", cities, year, month, day)
            daily_df = daily_df.rename(columns={"daily_avg_score": date})
            if result_df is None:
                result_df = daily_df
            else:
                result_df = result_df.merge(daily_df, on="city", how="outer", suffixes=('_x', '_y'))
                result_df["num_posts"] = result_df["num_posts_x"].fillna(0) + result_df["num_posts_y"].fillna(0)
                result_df.drop(columns=["num_posts_x", "num_posts_y"], inplace=True)
    result_df.drop(columns=["num_posts"], inplace=True)
    graph_df = result_df.T
    graph_df.rename(columns=graph_df.iloc[0], inplace=True)
    graph_df.drop(graph_df.index[0], inplace=True)
    graph_df.to_csv("".join(["../output/sentiment_graph_by_region/"] + cities + ["_year"]))
    return graph_df

In [ ]:
top_5_us_cities = ['Los Angeles', 'Harris', 'Cook', 'New York', 'Orange']

get_daily_sent_avg_n_numpost("United States", top_5_us_cities, 2019, 1, 10)

In [ ]:
graph_us_2019_df = generate_daily_avg_df_by_year("United States", top_5_us_cities, 2019)

# Removing not good data

## 2019 

In [ ]:
corrupted_files_2019_geo = pd.read_csv("../output/missing_file_report/corrupted_files_2019_geography.csv")

corrupted_files_2019_sent = pd.read_csv("../output/missing_file_report/corrupted_files_2019_sentiment.csv")

In [ ]:
corrupted_files_2019_geo.drop(columns="Unnamed: 0", inplace=True)
corrupted_files_2019_sent.drop(columns="Unnamed: 0", inplace=True)

In [ ]:
def get_date(file_name, category):
    if category == "geography":
        return "_".join(file_name.split("/")[5].split(".")[0].split("_")[1:4])
    else:
        return "_".join(file_name.split("/")[5].split(".")[0].split("_")[2:5])
    

In [ ]:
corrupted_files_2019_geo['dates'] = corrupted_files_2019_geo['corrupted_files'].apply(lambda x: get_date(x, "geography"))
corrupted_files_2019_sent['dates'] = corrupted_files_2019_sent['corrupted_files'].apply(lambda x: get_date(x, "sentiment"))

In [ ]:
corrupted_dates = set(corrupted_files_2019_sent['dates'].unique()) | set(corrupted_files_2019_geo['dates'].unique())

In [ ]:
corrupted_dates.add('2019_11_16') # to be fixed later

In [ ]:
year_df = None

for month in range(1, 13):
    month_df = pd.read_csv("".join(["../output/sentiment_graph_by_region/us_top5_cities_2019_", str(month), ".csv"]))
    month_df = month_df.rename(columns={"Unnamed: 0":"date"})
    if year_df is None:
        year_df = month_df
    else:
        year_df = pd.concat([year_df, month_df], ignore_index=True)

year_df

In [ ]:
top_5_us_cities = ['Los Angeles', 'Harris', 'Cook', 'New York', 'Orange']

for city in top_5_us_cities:
    year_df[city] = np.where(year_df['date'].isin(corrupted_dates), np.nan, year_df[city])

# year_df["Cook"] = np.where(year_df['date'].isin(corrupted_dates), np.nan, year_df["Cook"])

In [ ]:
for city in top_5_us_cities:
    # df = pd.read_csv("".join(["../output/sentiment_graph_by_region/", city, "_2019_1.csv"]))
    base = dt(2020, 1, 1)
    numdays = 365
    x = [base + timedelta(days=x) for x in range(numdays)]
    y = year_df[city]

    plt.plot(x, y)
    
    plt.title("".join(["Daily Average Sentiment of Posts in ", city, " County, 2019"]))
    plt.xticks(rotation = 45)
    plt.xlabel("Dates")
    plt.ylabel("Daily Average Sentiment")
    # plt.legend(bbox_to_anchor=(1.6, 1.0), loc='upper right')

    plt.show()
    # plt.savefig("".join(["../output/sentiment_graph_by_region/", city, "_2019_graph.png"]))